In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_blog_item(url) :
    tmp = url.split("/")
    url = f'https://blog.naver.com/PostView.naver?blogId={tmp[-2]}&logNo={tmp[-1]}'
    res = requests.get(url)
    nick = tmp[-2]
    soup = BeautifulSoup(res.text, "html.parser")
    
    if soup.select_one(".se-title-text") :
        title = soup.select_one(".se-title-text").text
        date = soup.select_one(".se_publishDate").text
        content = soup.select_one(".se-main-container").text
    else :
        title = soup.select_one(".se_title h3").text
        date = soup.select_one(".se_publishDate").text
        content = soup.select(".se_component_wrap")[1].text
        
    title = title.replace("\n", "").replace("\u200b", "").replace("\xa0", "").replace("\t", "").replace("\r","")
    content = content.replace("\n", "").replace("\u200b", "").replace("\xa0", "").replace("\t", "").replace("\r","")
        
    

    return (title, nick, date, content, url)

In [57]:
def get_naver_blog(keyword, startdate, enddate, to_csv=False) :
    ret = []
    for d in pd.date_range(startdate, enddate, freq='D') :
        print(d)
        #start = 1 # 1, 11, 21, 31, 41, 51, 61, 71, 81, 91
        page = 1
        while True :
            start = (page - 1) * 10 + 1
            url = f"https://s.search.naver.com/p/review/48/search.naver?ssc=tab.blog.all&api_type=8&query={keyword}&start={start}&nx_search_query=&nx_and_query=&nx_sub_query=&ac=1&aq=0&spq=0&sm=tab_opt&nso=so%3Add%2Cp%3Afrom{startdate.replace('.','')}to{enddate.replace('.','')}&prank=30&ngn_country=KR&lgl_rcode=09170128&fgn_region=&fgn_city=&lgl_lat=37.5278&lgl_long=126.9602&enlu_query=IggCADqCULhpAAAAj0RP67RmqTWRq3DkdYxhcV7F5RKpJAPG1d9ZSyMVgoc%3D&abt=&retry_count=0"
            res = requests.get(url)
            li = eval(res.text)['contents']
        
        ret = []
        
        while True :
            res = requests.get(url)
            res_dic = eval(res.text)
            soup = BeautifulSoup(res_dic['contents'], 'html.parser')
            if res_dic['nextUrl'] == "" :
                break
            url = res_dic['nextUrl']
        
            for item in soup.select('.title_area > a') :
                try :
                    ret.append(get_blog_item(item['href']))
                except :
                    print(item['href'])
            
        pd.DataFrame(ret, columns=['title', 'nick', 'date', 'content', 'url'])

In [58]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

keyword="티니핑"
startdate="2024.08.20"
enddate="2024.08.26"

def get_naver_news(keyword, startdate, enddate, to_csv=False) :
    ret = []
    for d in pd.date_range(startdate, enddate, freq='D') :
        print(d)
        #start = 1 # 1, 11, 21, 31, 41, 51, 61, 71, 81, 91
        page = 1
        while True :
            start = (page - 1) * 10 + 1
            url = f"https://s.search.naver.com/p/newssearch/search.naver?de={d.strftime('%Y.%m.%d')}&ds={d.strftime('%Y.%m.%d')}&eid=&field=0&force_original=&is_dts=0&is_sug_officeid=0&mynews=0&news_office_checked=&nlu_query=&nqx_theme=&nso=%26nso%3Dso%3Add%2Cp%3Afrom{d.strftime('%Y%m%d')}to{d.strftime('%Y%m%d')}%2Ca%3Aall&nx_and_query=&nx_search_hlquery=&nx_search_query=&nx_sub_query=&office_category=0&office_section_code=0&office_type=0&pd=3&photo=0&query={keyword}&query_original=&service_area=0&sort=1&spq=0&start={start}&where=news_tab_api&nso=so:dd,p:from{d.strftime('%Y%m%d')}to{d.strftime('%Y%m%d')},a:all"
            res = requests.get(url)
            li = eval(res.text)['contents']
            
            if len(li) == 0 :
                break

            for item in li :
                soup = BeautifulSoup(item, 'html.parser')
                a_tags = soup.select("div.info_group a")
                if len(a_tags) == 2 :
                    try :
                        ret.append(get_blog_item(a_tags[1]['href']))
                    except :
                        print(a_tags[1]['href'])
                        
            page += 1
                    
    df = pd.DataFrame(ret, columns = ["title", "media", "date", "content", "url"])
    if to_csv:
        df.to_csv(f"{keyword}_{startdate.replace('.','')}_{enddate.replace('.','')}.csv",index=False)
    return df

In [59]:
get_naver_blog('키니핑','2024.08.25','2024.08.26',to_csv=True)

2024-08-25 00:00:00


KeyboardInterrupt: 